# 분석환경 설정

In [16]:
import requests
import json
import time
import pandas as pd
import numpy as np

In [2]:
# RATE LIMITS
# 20 requests every 1 seconds(s)
# 100 requests every 2 minutes(s)

# 플레이어 동선 정확도 높이기
# 각 라인별 zone을 좌표로 설정하기
# 죽은 챔피언 그 위치에서 좌표가 찍힘. 따라서 킬 로그 먼저 확인한 이후에, 부활하기까지의 시간이 정확히 1분 단위에 포함된다면 좌표를 찍지 말아야 함.

# 데이터 수집 함수

## 티어별 유저 기본 데이터 가져오기 함수
- https://developer.riotgames.com/apis#league-v4
- /lol/league/v4/entries/{queue}/{tier}/{division}

In [21]:
def get_player_by_tier(api_key, queue="RANKED_SOLO_5x5", tier="challenger", division=1, page=1):
    """
    티어별 리그데이터를 가져오는 함수입니다.
    
    api_key : Riot에서 발급받은 api key
    queue   : 솔로랭크인지, 듀오랭크인지와 같은 queue type
    tier    : 랭크 티어
    division: tier에서 더 나뉘는 티어 1~4
    page    : 데이터를 가져오려는 기록 페이지
    
    """
    high_tier = ["challenger", "grandmaster", "master"]
    
    # 마스터 이상
    if tier in high_tier:
        tier_api = f"{kr_api}/lol/league/v4/{tier}leagues/by-queue/{queue}?page={page}&api_key={api_key}"
        
    # 다이아 이하
    else:
        division = "I" * division
        tier_api = f"{kr_api}/lol/league/v4/entries/{queue}/{tier}/{division}?page={page}&api_key={api_key}"
        
    data      = requests.get(tier_api).json()
    league_df = pd.DataFrame(data)
    
    return league_df

## 소환사 데이터 가져오기 함수
https://developer.riotgames.com/apis#account-v1

In [4]:
def get_summoner_data(api_key, summoner_info, typ="id"):
    """
    소환사에 관한 정보를 가져오는 함수입니다.
    
    api_key    : Riot에서 발급받은 api key
    summoner_id: 소환사 puuid
    typ        : 어떤 인자값을 통해 데이터를 가져올 것인지 정합니다. (id, name)
    """
    if typ == "id":
        summoner_api  = f"{kr_api}/lol/summoner/v4/summoners/{summoner_info}?api_key={api_key}"
    elif typ == "name":
        summoner_api  = f"{kr_api}/lol/summoner/v4/summoners/by-name/{summoner_info}?api_key={api_key}"
        
    response      = requests.get(summoner_api)
    summoner_data = response.json()
    
    return summoner_data

## match 리스트 가져오기 함수
- https://developer.riotgames.com/apis#match-v5
- /lol/match/v5/matches/by-puuid/{puuid}/ids

In [5]:
def get_matches_by_summoner_puuid(api_key, summoner_puuid, start=0, count=20):
    """
    소환사의 puuid로 match list 가져오는 함수입니다.
    
    api_key       : Riot에서 발급받은 api key
    summoner_puuid: Riot에서 제공한 소환사의 고유 ID
    start         : 가져올 매치의 시작 index
    count         : 가져올 매치의 개수 (max: 0 to 100)
    """
    if count > 100:
        raise Exception("최대 100개까지 가져올 수 있습니다.")
        
    match_list_api = f"{asia_api}/lol/match/v5/matches/by-puuid/{summoner_puuid}/ids?start={start}&count={count}&api_key={api_key}"
    response       = requests.get(match_list_api)
    match_list     = response.json()
    
    return match_list

## match 데이터 가져오기 함수
- https://developer.riotgames.com/apis#match-v5/GET_getMatch
- /lol/match/v5/matches/{matchId}

In [13]:
def get_match_by_match_id(api_key, match_id):
    """
    api_key : 발급받은 api key
    match_id: match id
    """
    match_data_api   = f"{asia_api}/lol/match/v5/matches/{match_id}?api_key={api_key}"
    response         = requests.get(match_data_api)
    match_basic_data = response.json()
    
    return match_basic_data

## 타임라인 match 데이터 가져오기 함수

In [7]:
def get_timeline_match_by_match_id(api_key, match_id):
    timeline_match_api = f"{asia_api}/lol/match/v5/matches/{match_id}/timeline?api_key={api_key}"
    response           = requests.get(timeline_match_api)
    time_match         = response.json()
    
    return time_match

## 챔피언 데이터 가져오기 함수

In [8]:
def get_all_champion_data(version="11.19.1"):
    champion_api = f"https://ddragon.leagueoflegends.com/cdn/{version}/data/ko_KR/champion.json"
    response = reuqests.get(champion_api)
    return response.json()

# 데이터 가져오기

- 20 requests every 1 seconds(s)
- 100 requests every 2 minutes(s)

In [9]:
# # 리그에 존재하는 유저 데이터 가져오기 (챌 300개, 그마 700개, 마스터 3400개)
# league_df = get_player_by_tier(api_key, tier="challenger", page=1)

# # 소환사 데이터 가져오기
# summoner_id    = league_df["entries"][0]["summonerId"]
# summoner_data  = get_summoner_data(api_key, summoner_id)
# summoner_puuid = summoner_data["puuid"]

# # match_list 가져오기
# match_list = get_matches_by_summoner_puuid(api_key, summoner_puuid, start=0, count=20)

# # 기본 match 데이터 가져오기
# match_num = 3
# match_id  = match_list[match_num]
# match_basic_data = get_match_by_match_id(api_key, match_id)

# # timeline match 데이터 가져오기
# time_match = get_timeline_match_by_match_id(api_key, match_id)
# time_match

In [13]:
basic_matches = {}
time_matches  = {}
page        = 8
high_tier   = ["challenger", "grandmaster", "master"]

challenger_df  = get_player_by_tier(api_key, tier="challenger", page=page)
grandmaster_df = get_player_by_tier(api_key, tier="grandmaster", page=page)
master_df      = get_player_by_tier(api_key, tier="master", page=page)

league_df = challenger_df.append([grandmaster_df, master_df]).reset_index(drop=True)

In [10]:
count = 0
for i in league_df.index:    
    # 소환사 데이터 가져오기
    pause_sign = 0
    
    summoner_id    = league_df["entries"][i]["summonerId"]
    summoner_data  = get_summoner_data(api_key, summoner_id)
    summoner_puuid = summoner_data["puuid"]

    # match_list 가져오기
    match_list = get_matches_by_summoner_puuid(api_key, summoner_puuid, start=0, count=30)

    # 기본 match 데이터 가져오기
    for match_id in match_list:            
        # 기본 매치 데이터 가져오기
        match_basic_data = get_match_by_match_id(api_key, match_id)

        # timeline match 데이터 가져오기
        time_match = get_timeline_match_by_match_id(api_key, match_id)

        # 데이터 저장하기
        basic_matches[match_id] = match_basic_data
        time_matches[match_id]  = time_match
        
        if count % 100 == 0:
            print(count)
        
        # 1000개만 갖고오기
        if count == 1000:
            pause_sign = 1
            break
        count += 1
        
    if pause_sign == 1:
        break

In [23]:
with open("data/basic_matches_1000.json", "w") as file:
    json.dump(basic_matches, file)
    
with open("data/time_matches_1000.json", "w") as file:
    json.dump(time_matches, file)